In [1]:
import websocket
import json

In [13]:
ws = websocket.WebSocket()
ws.connect('wss://plankton-app-5rfza.ondigitalocean.app/websites')
#ws.send("Hello, Server")
ws.send(json.dumps({'id': 'betway.com', 'url': 'https://betway.com/Pe', 'scraping': False}))
ws.recv()
ws.recv()

'{"status": "ok"}'